**15.2-1<br>Find an optimal parenthesization of a matrix-chain product whose sequence of dimension is $\langle 5,10,3,12,5,50,6\rangle$.**

We can use the `matrix_chain_order` and `print_optimal_parens` we defined in *15.2_Matrix_chain_multiplication.ipynb*, and substitute `p=np.array([5,10,3,12,5,50,6])`. 

In [19]:
import numpy as np
p=np.array([5,10,3,12,5,50,6])

def matrix_chain_order(p):
    # n=p.length-1
    n=len(p)-1
    #let m[1...n,1...n] to store the m[i,j] cost
    m=np.zeros((n+1,n+1))
    #let s[1...n-1,2...n] that records which index of k achieved the optimal cost in computing m[i,j]
    s=np.zeros((n,n+1),dtype=int)
    for l in range(2,n+1): #l is the chain length, l=j-i+1
        for i in range(1,n-l+2): #possible range of i, min=1, max=n-1
            j=i+l-1              #because l=j-i+1, min=2,max=n
            m[i,j]=np.inf        #set infinity as sentinel for m[i,j]
            for k in range(i,j):
                q=m[i,k]+m[k+1,j]+p[i-1]*p[k]*p[j]
                if q<m[i,j]:
                    m[i,j]=q
                    s[i,j]=k
    return m,s

def print_optimal_parens(s,i,j):
    if i==j:
        print ('A'+str(i),end='')
    else:
        print ('(', end='')
        print_optimal_parens(s,i,s[i,j]) #k=s[i,j], split product Ai...Ak
        print_optimal_parens(s,s[i,j]+1,j) #k+1=s[i,j]+1, split product A(k+1)...Aj 
        print (')',end='')
matrix_chain_order(p)[1]
#print_optimal_parens(matrix_chain_order(p)[1],1,6)    

array([[0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 2, 2, 4, 2],
       [0, 0, 0, 2, 2, 2, 2],
       [0, 0, 0, 0, 3, 4, 4],
       [0, 0, 0, 0, 0, 4, 4],
       [0, 0, 0, 0, 0, 0, 5]])

**15.2<br>Give a recursive algorithm MATRIX-CHAIN-MULTIPLY(A,s,i,j) that actually performs the optimal matrix-chain multiplication, given the sequence of matrices $\langle A_1,A_2,...,A_n\rangle$, the $s$ table computed by MATRIX-CHAIN-ORDER, and the indices i and j. (The initial call would be MATRIX-CHAIN-MULTIPLY(A,s,1,n).)**

1. Construct function `matrix_multiply` based on the pseudocodes on *p371*
2. `matrix_chain_multiply` takes the input:
    * `A` is a 3-D numpy array of chain $A_1A_2...A_n$, whose length is $n$
    * `s` comes from `matrix_chain_order(p)[1]` from *15.1*
    * the initial values of $(i,j)$ are $(1,n)$
3. Recursive call in `matrix_chain_multiply`:
    * recursion (Line 6-7): the original input chain $A_1A_2...A_n$ keeps spliting at the position `k=s[i,j]`
    * base case no. 1(Line 2-3): when the the chain is split to a point that its length is 1 (i.e. it contains a single matrix), it returns `A[0]`, the matrix itself  
    * base case no. 2(Line 4-5): when the the chain is split to a point that its length is 2 (i.e. $j=i+1$), we can apply `matrix_multiply(A[0],A[1]`
    * return (Line 8): the final result is the recursive multiplication of left-split ($A_1A_2...A_k$) and right-split ($A_{k+1}A_{k+2}...A_n$)
    
$*$Notice that in Python the slicing of an array `arr[a:b]` includes index `a` but excludes `b`, in Line 4-5 we finetune the indexing of A inside the recursive call, so each is $-1$ of the original index.

In [23]:
def matrix_chain_multiply(A,s,i,j):
    if len(A)==1:
        return A[0]
    if len(A)==2: 
        return matrix_multiply(A[0],A[1])
    X=matrix_chain_multiply(A[i-1:s[i,j]],s,i,j) 
    Y=matrix_chain_multiply(A[s[i,j]:],s,i,j)
    return matrix_multiply(X,Y)

def matrix_multiply(A,B):
    A_row=A.shape[0]
    A_col=A.shape[1]
    B_row=B.shape[0]
    B_col=B.shape[1]
    if A_col !=B_row:
        raise Exception('A and B are incompatible!')
    C=np.zeros((A_row,B_col))
    for i in range(A_row):
        for j in range(B_col):
            for k in range(A_col):
                C[i,j]=C[i,j]+A[i,k]*B[k,j]
    return C

p2=np.array([3,2,2,4,3,7,7])
# A1-A6 according to p2
arr1=np.arange(6).reshape(3,2)
arr2=np.arange(4).reshape(2,2)
arr3=np.arange(8).reshape(2,4)
arr4=np.arange(12).reshape(4,3)
arr5=np.arange(21).reshape(3,7)
arr6=np.eye(7)
arr=np.array([arr1,arr2,arr3,arr4,arr5,arr6])

s2=matrix_chain_order(p2)[1]
matrix_chain_multiply(arr,s2,1,6)

array([[ 11676.,  13188.,  14700.,  16212.,  17724.,  19236.,  20748.],
       [ 41356.,  46708.,  52060.,  57412.,  62764.,  68116.,  73468.],
       [ 71036.,  80228.,  89420.,  98612., 107804., 116996., 126188.]])